In [4]:
from music21 import converter, key

def get_song_key(mp3_file):
    # Convert MP3 to Music21 Stream
    stream = converter.parse(mp3_file)
    
    # Get key signature
    ks = stream.analyze('key')
    
    return ks

# Example usage
mp3_file_path = "/Users/zac/Downloads/Choo Lo.mp3"
song_key = get_song_key(mp3_file_path)
print("Key of the song:", song_key.tonic, song_key.mode)


ConverterFileException: cannot find a format extensions for: /Users/zac/Downloads/Choo Lo.mp3

In [1]:
import aubio

def get_song_key(mp3_file):
    # Create aubio source
    audio_source = aubio.source(mp3_file)

    # Create aubio pitch object
    pitch_o = aubio.pitch("yin", audio_source.samplerate, aubio.source.get_channels(audio_source))

    # Lists to store detected pitches
    detected_pitches = []

    # Total number of frames read
    total_frames = 0

    while True:
        samples, read = audio_source()
        pitch = pitch_o(samples)[0]
        detected_pitches.append(pitch)
        total_frames += read
        if read < audio_source.hop_size:
            break

    # Find the most common pitch
    most_common_pitch = max(set(detected_pitches), key=detected_pitches.count)

    # Map pitch to key
    pitch_to_key_map = {
        57: "A",
        58: "A#",
        59: "B",
        60: "C",
        61: "C#",
        62: "D",
        63: "D#",
        64: "E",
        65: "F",
        66: "F#",
        67: "G",
        68: "G#",
    }

    # Assuming A=57, determine key based on most common pitch
    key_number = (most_common_pitch - 57) % 12
    key = pitch_to_key_map[key_number]

    return key

# Example usage
mp3_file_path = "/Users/zac/Downloads/Choo Lo.mp3"
song_key = get_song_key(mp3_file_path)
print("Key of the song:", song_key)


ValueError: input size of pitch should be 2, not 512

In [2]:
import librosa
import numpy as np

def get_song_key(mp3_file):
    # Load the audio file
    y, sr = librosa.load(mp3_file)

    # Estimate the global key
    key, key_strength = librosa.key.global_key(y)

    # Map the key index to key name
    key_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

    # Get the key name
    key_name = key_names[key]

    return key_name

# Example usage
mp3_file_path = "/Users/zac/Downloads/Choo Lo.mp3"
song_key = get_song_key(mp3_file_path)
print("Key of the song:", song_key)


AttributeError: No librosa attribute key

In [3]:
import librosa as lb


y , sr = lb.load(mp3_file_path)

In [5]:
sr

22050

In [8]:
lb.feature.tempo(y=y, sr=sr)

array([143.5546875])

In [ ]:

from typing import Any
from typing_extensions import Unpack
from pydantic import BaseModel, ConfigDict

class SongAnalysis(BaseModel):
    def __init_subclass__(cls, **kwargs: *ConfigDict):
        return super().__init_subclass__(**kwargs)
    

    


    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return super().__call__(*args, **kwds)


In [ ]:
# https://github.com/jackmcarthur/musical-key-finder/blob/master/keyfinder.py

In [11]:
import os
import librosa
import numpy as np
import pandas as pd

# Function to extract features from an audio file
def extract_features(file_path):
    try:
        # Load audio file
        y, sr = librosa.load(file_path)

        # Extract features
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
        rmse = np.mean(librosa.feature.rms(y=y))
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=y))
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)

        # Create dictionary of features
        features = {
            'tempo': tempo,
            'chroma_stft': chroma_stft,
            'rmse': rmse,
            'spectral_centroid': spectral_centroid,
            'spectral_bandwidth': spectral_bandwidth,
            'spectral_flatness': spectral_flatness
        }
        for i, mfcc in enumerate(mfccs):
            features[f'mfcc{i+1}'] = mfcc

        return features

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Function to analyze a directory of audio files and create a database
def analyze_directory(directory):
    # Create empty list to store features
    all_features = []

    # Iterate over files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.wav'):
            file_path = os.path.join(directory, filename)
            features = extract_features(file_path)
            if features:
                features['filename'] = filename
                all_features.append(features)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_features)

    # Save DataFrame to CSV file
    df.to_csv('music_features.csv', index=False)

# Example usage
directory = '/Users/zac/Codes/Music_Project/GIT_HUB/Musis_Recommendation_Engine/exploration/song_analysis/'
analyze_directory(directory)


In [13]:
def calculate_danceability(y, sr):
    # Compute the onset envelope
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    
    # Aggregate by mean
    mean_onset_env = np.mean(onset_env)
    
    # Danceability is higher if the rhythm is strong and stable
    return mean_onset_env


def calculate_liveliness(y):
    # Calculate spectral centroid variation
    centroid_var = np.var(librosa.feature.spectral_centroid(y=y))
    
    # Higher variation indicates more liveliness
    return centroid_var

def calculate_acousticness(y, sr):
    # Calculate spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr))
    
    # Higher spectral contrast indicates less acousticness
    return 1 - contrast


def estimate_key(y, sr):
    # Calculate the chromagram
    chroma = librosa.feature.chroma_cens(y=y, sr=sr)
    
    # Find the most prominent key
    key = np.argmax(np.mean(chroma, axis=1))
    
    return key


In [14]:
calculate_acousticness(y ,sr)

-22.98308975971634

In [15]:
calculate_danceability(y,sr)

1.104514

In [16]:
calculate_liveliness(y)

478552.3426545928

In [17]:
estimate_key(y, sr)

11

In [20]:
def estimate_key(y, sr):
    # Calculate the chromagram
    chroma = librosa.feature.chroma_cens(y=y, sr=sr)
    print(chroma)
    
    # Define key names
    key_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    
    # Find the most prominent key
    key_index = np.argmax(np.mean(chroma, axis=1))
    print(key_index)
    
    # Get the corresponding key name
    key_name = key_names[key_index]
    
    return key_name


In [21]:
estimate_key(y, sr)

[[0.4035483  0.396904   0.3891428  ... 0.11983906 0.12707949 0.134336  ]
 [0.40562487 0.39775664 0.38871878 ... 0.         0.         0.        ]
 [0.31019384 0.3012065  0.29175636 ... 0.         0.         0.        ]
 ...
 [0.15925007 0.17378949 0.18834077 ... 0.         0.         0.        ]
 [0.24439164 0.25059277 0.2572321  ... 0.3777264  0.38432527 0.3907277 ]
 [0.25196087 0.25928444 0.2666341  ... 0.9133812  0.9104386  0.90738636]]
11


'B'

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display

# class that uses the librosa library to analyze the key that an mp3 is in
# arguments:
#     waveform: an mp3 file loaded by librosa, ideally separated out from any percussive sources
#     sr: sampling rate of the mp3, which can be obtained when the file is read with librosa
#     tstart and tend: the range in seconds of the file to be analyzed; default to the beginning and end of file if not specified
class Tonal_Fragment(object):
    def __init__(self, waveform, sr, tstart=None, tend=None):
        self.waveform = waveform
        self.sr = sr
        self.tstart = tstart
        self.tend = tend
        
        if self.tstart is not None:
            self.tstart = librosa.time_to_samples(self.tstart, sr=self.sr)
        if self.tend is not None:
            self.tend = librosa.time_to_samples(self.tend, sr=self.sr)
        self.y_segment = self.waveform[self.tstart:self.tend]
        self.chromograph = librosa.feature.chroma_cqt(y=self.y_segment, sr=self.sr, bins_per_octave=24)
        
        # chroma_vals is the amount of each pitch class present in this time interval
        self.chroma_vals = []
        for i in range(12):
            self.chroma_vals.append(np.sum(self.chromograph[i]))
        pitches = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
        # dictionary relating pitch names to the associated intensity in the song
        self.keyfreqs = {pitches[i]: self.chroma_vals[i] for i in range(12)} 
        
        keys = [pitches[i] + ' major' for i in range(12)] + [pitches[i] + ' minor' for i in range(12)]

        # use of the Krumhansl-Schmuckler key-finding algorithm, which compares the chroma
        # data above to typical profiles of major and minor keys:
        maj_profile = [6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88]
        min_profile = [6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17]

        # finds correlations between the amount of each pitch class in the time interval and the above profiles,
        # starting on each of the 12 pitches. then creates dict of the musical keys (major/minor) to the correlation
        self.min_key_corrs = []
        self.maj_key_corrs = []
        for i in range(12):
            key_test = [self.keyfreqs.get(pitches[(i + m)%12]) for m in range(12)]
            # correlation coefficients (strengths of correlation for each key)
            self.maj_key_corrs.append(round(np.corrcoef(maj_profile, key_test)[1,0], 3))
            self.min_key_corrs.append(round(np.corrcoef(min_profile, key_test)[1,0], 3))

        # names of all major and minor keys
        self.key_dict = {**{keys[i]: self.maj_key_corrs[i] for i in range(12)}, 
                         **{keys[i+12]: self.min_key_corrs[i] for i in range(12)}}
        
        # this attribute represents the key determined by the algorithm
        self.key = max(self.key_dict, key=self.key_dict.get)
        self.bestcorr = max(self.key_dict.values())
        
        # this attribute represents the second-best key determined by the algorithm,
        # if the correlation is close to that of the actual key determined
        self.altkey = None
        self.altbestcorr = None

        for key, corr in self.key_dict.items():
            if corr > self.bestcorr*0.9 and corr != self.bestcorr:
                self.altkey = key
                self.altbestcorr = corr
                
    # prints the relative prominence of each pitch class            
    def print_chroma(self):
        self.chroma_max = max(self.chroma_vals)
        for key, chrom in self.keyfreqs.items():
            print(key, '\t', f'{chrom/self.chroma_max:5.3f}')
                
    # prints the correlation coefficients associated with each major/minor key
    def corr_table(self):
        for key, corr in self.key_dict.items():
            print(key, '\t', f'{corr:6.3f}')
    
    # printout of the key determined by the algorithm; if another key is close, that key is mentioned
    def print_key(self):
        print("likely key: ", max(self.key_dict, key=self.key_dict.get), ", correlation: ", self.bestcorr, sep='')
        if self.altkey is not None:
                print("also possible: ", self.altkey, ", correlation: ", self.altbestcorr, sep='')
    
    # prints a chromagram of the file, showing the intensity of each pitch class over time
    def chromagram(self, title=None):
        C = librosa.feature.chroma_cqt(y=self.waveform, sr=sr, bins_per_octave=24)
        plt.figure(figsize=(12,4))
        librosa.display.specshow(C, sr=sr, x_axis='time', y_axis='chroma', vmin=0, vmax=1)
        if title is None:
            plt.title('Chromagram')
        else:
            plt.title(title)
        plt.colorbar()
        plt.tight_layout()
        plt.show()

In [23]:
hey = Tonal_Fragment(waveform=y, sr = sr)

In [25]:
hey.print_key()

likely key: B major, correlation: 0.724
also possible: E major, correlation: 0.689


In [26]:
hey.print_chroma()

C 	 0.512
C# 	 0.434
D 	 0.350
D# 	 0.403
E 	 0.659
F 	 0.432
F# 	 0.575
G 	 0.421
G# 	 0.441
A 	 0.563
A# 	 0.434
B 	 1.000


In [27]:
y_harm = librosa.effects.harmonic(y)

In [28]:
new = Tonal_Fragment(y_harm, sr)

In [29]:
new.print_key()

likely key: B major, correlation: 0.683
also possible: E major, correlation: 0.646


In [12]:

gpt_list = """1. Chill Vibes: relaxation : stress
2. Uplifting Anthems: happiness : sadness
3. Melancholic Melodies: sadness : melancholy
4. Power Ballads: empowerment : low self-esteem
5. Jazzy Nights: nostalgia : loneliness
6. Dancefloor Hits: excitement : boredom
7. Rainy Day Tunes: introspection : gloominess
8. Acoustic Serenade: comfort : heartache
9. Inspirational Beats: motivation : discouragement
10. Indie Gems: uniqueness : feeling unnoticed
11. Feel-Good Favorites: positivity : negativity
12. Summer Breeze: warmth : coldness
13. Heartfelt Ballads: emotion : numbness
14. Road Trip Jams: adventure : monotony
15. Soulful Grooves: depth : shallowness
16. Electro Pop Fun: energy : lethargy
17. Rhythmic Rapture: movement : stillness
18. Healing Harmonies: healing : pain
19. Mellow Moods: calmness : agitation
20. Reggae Relaxation: tranquility : restlessness
21. Classic Rock Revival: nostalgia : disconnection
22. Folk Feelings: authenticity : falseness
23. Romantic Rhythms: love : heartbreak
24. Hip Hop Therapy: empowerment : frustration
25. Ambient Escapes: peace : chaos
26. Bluesy Resonance: introspection : despair
27. Country Comfort: familiarity : estrangement
28. Pop Perfection: catchiness : indifference
29. Smooth Jazz Soiree: sophistication : rawness
30. Motivational Mix: determination : doubt
31. Electronic Dreams: imagination : stagnation
32. Lyrical Exploration: storytelling : emptiness
33. Latin Fiesta: celebration : sorrow
34. Feel-Good Funk: groove : stiffness
35. Piano Ponderings: reflection : distraction
36. Worldly Wonders: diversity : monotony
37. Ambient Bliss: serenity : restlessness
38. Soulful Reflections: introspection : confusion
39. Dreamy Delights: whimsy : seriousness
40. Rock Revival: rebellion : conformity
41. Gospel Glory: faith : doubt
42. R&B Rejuvenation: sensuality : inhibition
43. Jazz Journey: improvisation : rigidity
44. Indie Introspection: authenticity : superficiality
45. Classical Calm: elegance : clumsiness
46. Electronic Enchantment: euphoria : depression
47. Folklore Feels: storytelling : detachment
48. Nostalgic Nights: reminiscence : detachment
49. Eclectic Excursions: diversity : sameness
50. Harmonic Hope: optimism : pessimism
51. Laid-back Lounge: ease : tension
52. Melodic Memories: nostalgia : forgetfulness
53. Sentimental Serenade: sentimentality : apathy
54. Symphony of Serenity: tranquility : agitation
55. Acoustic Affection: intimacy : distance
56. Disco Fever: joy : apathy
57. Alternative Avenue: individuality : conformity
58. Blissful Beats: euphoria : despondency
59. Coffeehouse Crooners: intimacy : loneliness
60. Ethereal Echoes: otherworldliness : mundanity
61. Groovy Goodness: rhythm : discord
62. Harmonious Heights: unity : division
63. Vibrant Voices: expression : repression
64. Retro Rewind: nostalgia : detachment
65. Sentimental Sojourn: sentimentality : indifference
66. Ambient Adventure: exploration : stagnation
67. Ballad Bouquet: emotion : numbness
68. Dynamic Dreamscape: imagination : reality
69. Enchanted Emotions: wonder : skepticism
70. Jazzed-Up Jams: improvisation : predictability
71. Lively Landscapes: energy : lethargy
72. Poetic Portraits: lyricism : literalism
73. Psychedelic Symphony: exploration : conventionality
74. Radiant Rhythms: vibrancy : dullness
75. Sensational Soundscape: sensation : numbness
76. Serene Sonnets: tranquility : turmoil
77. Spiritual Solace: connection : isolation
78. Sultry Sensations: sensuality : inhibition
79. Tender Tunes: gentleness : harshness
80. Tranquil Trails: peace : unrest
81. Upbeat Utopia: positivity : negativity
82. Whimsical Wanderlust: imagination : reality
83. Zen Zone: mindfulness : distraction
84. Ambient Atmosphere: serenity : chaos
85. Ballroom Bliss: elegance : awkwardness
86. Cosmic Connections: universality : individualism
87. Dreamy Disposition: whimsy : practicality
88. Groove Garden: rhythm : dissonance
89. Harmony Haven: unity : discord
90. Jazzy Junction: improvisation : rigidity
91. Luminescent Lullabies: brightness : darkness
92. Melodic Meadows: tranquility : turbulence
93. Peaceful Paradise: serenity : unrest
94. Rhythmic Reverie: movement : stillness
95. Sonic Sanctuary: refuge : exposure
96. Tranquil Tempos: calmness : agitation
97. Velvet Voices: intimacy : distance
98. Whispering Winds: softness : harshness
99. Zenith Zone: elevation : stagnation
100. Ethereal Embrace: otherworldliness : mundanity"""

search_terms = gpt_list.split("\n")
final_list = []

for terms in search_terms:
    final = terms.split(".")[-1]
    final_list.append(final.strip())


['Chill Vibes: relaxation : stress',
 'Uplifting Anthems: happiness : sadness',
 'Melancholic Melodies: sadness : melancholy',
 'Power Ballads: empowerment : low self-esteem',
 'Jazzy Nights: nostalgia : loneliness',
 'Dancefloor Hits: excitement : boredom',
 'Rainy Day Tunes: introspection : gloominess',
 'Acoustic Serenade: comfort : heartache',
 'Inspirational Beats: motivation : discouragement',
 'Indie Gems: uniqueness : feeling unnoticed',
 'Feel-Good Favorites: positivity : negativity',
 'Summer Breeze: warmth : coldness',
 'Heartfelt Ballads: emotion : numbness',
 'Road Trip Jams: adventure : monotony',
 'Soulful Grooves: depth : shallowness',
 'Electro Pop Fun: energy : lethargy',
 'Rhythmic Rapture: movement : stillness',
 'Healing Harmonies: healing : pain',
 'Mellow Moods: calmness : agitation',
 'Reggae Relaxation: tranquility : restlessness',
 'Classic Rock Revival: nostalgia : disconnection',
 'Folk Feelings: authenticity : falseness',
 'Romantic Rhythms: love : heartbre